In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
classifer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
# results = classifer(["We are happy to see u", "I hate eatting veggies"])

# Using HuggingFace For Sentiment Analysis..
## Data taken from Twitter

In [5]:
import tweepy as twt
from textblob import TextBlob
# from wordcloud import WordCloud
import pandas as pd
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [6]:
loginDetails = pd.read_csv('Login.csv')

In [7]:
# The Twitter API credentials
consumerKey = loginDetails['key'][0]
consumerSecret = loginDetails['key'][1]
accessToken = loginDetails['key'][2]
accessTokenSecret = loginDetails['key'][3]


In [8]:
# Auth Object
authenticate = twt.OAuthHandler(consumerKey, consumerSecret)

# Set access token and access token secret
authenticate.set_access_token(accessToken, accessTokenSecret)

# Create API object
api = twt.API(authenticate, wait_on_rate_limit=True)

In [9]:
# Extract 100 tweets from Twitter user
posts = api.user_timeline(id = '@owusu2kelvin', count=20, lang = 'en', tweet_mode = 'extended')

# Show tweets
print("Showing tweets:" + '\n')
n = 1
for tweet in posts[0:5]:
    print(str(n) + ') '+tweet.full_text + "\n")
    n=n+1

Unexpected parameter: id
Unexpected parameter: lang


Showing tweets:

1) RT @GrizzOnBally: ARE YOU KIDDING ME JA MORANT?! 🤯

@JaMorant | #NBAAllStar https://t.co/naMJocd3Pj

2) RT @bluewirepods: The Splash Bros are back 😈 https://t.co/NlLvUAqxoE

3) RT @EmiratesFACup: 𝗕𝗔𝗖𝗞-𝗛𝗘𝗘𝗟 𝗚𝗢𝗔𝗟 🔥

Just Firmino things @LFC 🇧🇷

#EmiratesFACup https://t.co/f6u45MpUWd

4) RT @SportsCenter: The rest of the league when Klay's returning to the 29-9 Warriors: https://t.co/6u9WOikNLs

5) RT @KingJosiah54: Klay coming back to the Warriors https://t.co/RhWbNUWCXo



In [10]:
# Create dataframe with column called tweets
tweets_df = pd.DataFrame([tweet.full_text for tweet in posts], columns=['Tweets'])

tweets_df.head()

,Tweets
0,RT @GrizzOnBally: ARE YOU KIDDING ME JA MORANT...
1,RT @bluewirepods: The Splash Bros are back 😈 h...
2,RT @EmiratesFACup: 𝗕𝗔𝗖𝗞-𝗛𝗘𝗘𝗟 𝗚𝗢𝗔𝗟 🔥\n\nJust Fi...
3,RT @SportsCenter: The rest of the league when ...
4,RT @KingJosiah54: Klay coming back to the Warr...


In [11]:
# Cleaning the text using a function
def cleanTxt(text):
    text =re.sub(r'@[A-Za-z0-9_]+', '', text) # Removing @mentions
    text = re.sub(r'#', '', text) # Removing Hashtags
    text = re.sub(r'RT[\s]+:', '', text) # Removing RT
    text = re.sub(r'https?:\/\/\S+', '', text) # Removing hyperlinks
    text = text.replace("\n","")
    
    return text

tweets_df['Tweets'] = tweets_df['Tweets'].apply(cleanTxt) # Cleaning the texts

tweets_df.head() # Showing the cleaned texts

,Tweets
0,ARE YOU KIDDING ME JA MORANT?! 🤯 | NBAAllStar
1,The Splash Bros are back 😈
2,𝗕𝗔𝗖𝗞-𝗛𝗘𝗘𝗟 𝗚𝗢𝗔𝗟 🔥Just Firmino things 🇧🇷Emirat...
3,The rest of the league when Klay's returning ...
4,Klay coming back to the Warriors


In [12]:
tweet_results = []
for tweet in tweets_df['Tweets']:
    tweet_results.append(str(tweet))

tweet_results

[' ARE YOU KIDDING ME JA MORANT?! 🤯 | NBAAllStar ',
 ' The Splash Bros are back 😈 ',
 ' 𝗕𝗔𝗖𝗞-𝗛𝗘𝗘𝗟 𝗚𝗢𝗔𝗟 🔥Just Firmino things  🇧🇷EmiratesFACup ',
 " The rest of the league when Klay's returning to the 29-9 Warriors: ",
 ' Klay coming back to the Warriors ',
 ' “I’m back.” -  Klay941 ',
 ' HIGH SCHOOLER Brandon Miller must’ve been studying Vince Carter vs. France dunks before this slam. 🤯(🎥:IG/black_mosez, H/T…',
 ' Storyline is a bit cliché but the fight scenes are dope!!!!',
 ' Moi!!!!',
 ' Who else can relate?🤦🏾\u200d♂️🤣😀PulseViral ',
 ' There’s optimism Golden Warriors guard Klay Thompson will make his return vs. the Cleveland Cavaliers on Sunday, sources tell…',
 " If you need us, we'll be watching this on repeat all day 🤩(via )",
 ' ELECTRIC. Andre Roberts 101-yard kick return TD BoltUp📺: DENvsLAC on CBS📱: NFL app ',
 ' AB straight up just left 😳(via )',
 ' DEROZAN. GAME. AGAIN. 😱(via )',
 ' ▫️ Lukaku unhappy ▫️ Eight points behind City ▫️ Injuries in defense piling up Chelsea fans:

In [13]:
tweet_results = classifer(tweet_results)

In [15]:
positive = 0
negative = 0
# print(tweet_results)
for result in tweet_results:
    if result['label'] == 'POSITIVE':
        positive += 1
    else:
        negative += 1

print(f"The total number of Negative Sentiments are {negative}.\nThe total number of Positive Sentiments are {positive}.")

[{'label': 'NEGATIVE', 'score': 0.9856570959091187}, {'label': 'POSITIVE', 'score': 0.893064558506012}, {'label': 'NEGATIVE', 'score': 0.6658878922462463}, {'label': 'POSITIVE', 'score': 0.6438055038452148}, {'label': 'POSITIVE', 'score': 0.9939017295837402}, {'label': 'POSITIVE', 'score': 0.8953263759613037}, {'label': 'NEGATIVE', 'score': 0.9948674440383911}, {'label': 'NEGATIVE', 'score': 0.9932460188865662}, {'label': 'POSITIVE', 'score': 0.9900117516517639}, {'label': 'NEGATIVE', 'score': 0.9960796236991882}, {'label': 'POSITIVE', 'score': 0.9989639520645142}, {'label': 'NEGATIVE', 'score': 0.9470311999320984}, {'label': 'NEGATIVE', 'score': 0.9524260759353638}, {'label': 'NEGATIVE', 'score': 0.990864098072052}, {'label': 'NEGATIVE', 'score': 0.9907160997390747}, {'label': 'NEGATIVE', 'score': 0.8513162136077881}, {'label': 'POSITIVE', 'score': 0.9937077760696411}, {'label': 'NEGATIVE', 'score': 0.9982025623321533}, {'label': 'POSITIVE', 'score': 0.9887125492095947}, {'label': 'PO